In [1]:
{-# LANGUAGE NoMonomorphismRestriction #-}
{-# LANGUAGE FlexibleInstances #-}
import Control.Monad

#  Interpreters for first-order languages

In usual, we prefer infix expressions like

In [2]:
1919 - (114 + 514)

1291

## Initial embedding

However, when the *initial* embedding encodes expression as a value of algebraic data type:

In [3]:
data Exp = Lit Int
    | Neg Exp
    | Add Exp Exp

The expression above is rewritten with such a prefix notation (sometimes also known as *Polish notation*):

In [4]:
x :: Exp
x = Add (Lit 1919) (Neg (Add (Lit 114) (Lit 514)))

We know the evaluation should be equal to the following prefix expression in Haskell:

In [5]:
(+) 1919 (negate ((+) 114 514))

1291

### Eval

Then, we create the first interpreter, *evaluator*, which interprets expression by case analysis, in other words, *pattern matching*:

In [6]:
eval:: Exp -> Int
eval (Lit n)     = n
eval (Neg e)     = - eval e
eval (Add e1 e2) = eval e1 + eval e2

Let's eval `x` above:

In [7]:
eval x

1291

## Final Embedding

We can embed expressions in a different way:

First, we introduce a representation type for an expression. In the case of `eval`, since the calculation result `Int`, the type used to represent it should also be defined as:

In [8]:
type Repr = Int

Now, use `Repr` as the *target* type for evaluation process, we can directly consider expressions as functions:

In [9]:
lit :: Int -> Repr
lit n = n

neg :: Repr -> Repr
neg e = - e

add :: Repr -> Repr -> Repr
add e1 e2 = e1 + e2

Functions are *compositional*. We can construct an expression represented by those functions and it will be immediately evaluated:

In [10]:
add (lit 1919) (neg (add (lit 114) (lit 514)))

1291

We call this metacircular embedding a *final embedding*, which is dual to *initial embedding*.

### Pretty Print

But when it comes to adding another interpretor, for example, *pretty-printer* to the language, initial embedding seems more *scalable*: we just need to write a new function in which applies new patch-matching process.

In [11]:
view:: Exp -> String
view (Lit n) = show n
view (Neg e) = "(-" ++ view e ++ ")"
view (Add e1 e2) = "(" ++ view e1 ++ " + " ++ view e2 ++ ")"

In [12]:
view x

"(1919 + (-(114 + 514)))"

The `view` interpreter 'evaluates' expressions in a very similar way to `eval`, except that the type of final results is `String` instead of `Int`.

In the final embedding, the evaluator interpreter is hardwired into an expression, which indicates it is impossible to interpret the expression in another way rather that `eval`.

We want the final embedding to support for multiple interpreters as well. One solution offered by Haskell is to write a [*type class*](https://www.haskell.org/tutorial/classes.html) that enables such a parametrization for final result type. 

We define a type class `ExpSYM`, in which every instance `repr` (i.e. final result type of an interpreter) should has the following properties (in this case, can be involved in those 3 functions):

In [13]:
class ExpSYM repr where
    lit :: Int -> repr
    neg :: repr -> repr
    add :: repr -> repr -> repr

In the case of `eval`, the result type is `Int`. So we will specify the how to implement`ExpSYM` instance for `Int` type:

In [14]:
instance ExpSYM Int where
    lit n = n
    neg e = - e
    add e1 e2 = e1 + e2

To map the result type `Int` to actual Haskell `Int` type (alright, it sounds awkward), we define `eval` function as:

In [15]:
eval :: Int -> Int
eval = id

Then we can `eval` the expression above:

In [16]:
eval $ add (lit 1919) (neg (add (lit 114) (lit 514)))

1291

A finally-encoded expression can have multiple ways to interpret. `eval` specifies one of them.

Unlike the initial interpreter:

```hs
eval :: Exp -> Int
```

Our final interpreter uses no pattern-matching. It means no syntax-dispatch overhead, which is similar to [threaded code](https://www.complang.tuwien.ac.at/forth/threaded-code.html) in some way.

`SYM` in `ExpSYM` (name of our type class) stands for *symantics*: the type class defines the syntax about how an expression is embedded (expression form), while an type instance specifies how to interpret the expression.

Now, adding more interpreters for an expression is possible: we may interpret it as a `String`, then we need to implement a pretty-printer as:

In [17]:
instance ExpSYM String where
    lit = show
    neg e = "(-" ++ e ++ ")"
    add e1 e2 = "(" ++ e1 ++ " + " ++ e2 ++ ")"

In [18]:
view :: String -> String
view = id

In [19]:
view $ add (lit 1919) (neg (add (lit 114) (lit 514)))

"(1919 + (-(114 + 514)))"

In the initial embedding, expressions (`Exp`) and interpreters (functions like `eval`,`view`) are totally ordinary and monomophic values, which are certainly first-class elements.

So we can store all expressions into one `List[Exp]` as:

In [20]:
lst :: [Exp]
lst = [Lit 114, Lit 514, Add (Lit 114) (Lit 514)]

So we can apply an interpreter on all of them using `fmap` like (we rename `eval` as `eval2`, for that `eval` for `Exp` is already redefined for `ExpSYM` above):

In [21]:
eval2:: Exp -> Int
eval2 (Lit n)     = n
eval2 (Neg e)     = - eval2 e
eval2 (Add e1 e2) = eval2 e1 + eval2 e2

fmap eval2 lst

[114,514,628]

Meanwhile, it seems impossible in the final embedding, in which expressions are polymorphic.

Those polymorphic expressions are not totally first-class: **storing them in data structures or passing them as arguments may cause the loss of polymorphism.**

Although sometimes it doesn't matter, for example, they can be collected in one list:

In [22]:
lst_sym = [lit 114, lit 514, add (lit 114) (lit 514)]

Line 1: Use camelCase
Found:
lst_sym = ...
Why not:
lstSym = ...

The type of `lst_sym` still cannot be exactly determined, all we can infer from it is that the list consists of type that satisfies `ExpSYM`. It means that polymorphism remains here.

In [23]:
:type lst_sym

lst_sym :: forall a. ExpSYM a => [a]

And then `fmap` on it:

In [24]:
fmap eval lst_sym

[114,514,628]

It resembles the method that specifies the type of a polymorphic value, since `eval` is defined as a simple `id` function for `ExpSYM` class:

In [25]:
[lit 114, lit 514, add (lit 114) (lit 514)]::[Int]

[114,514,628]

Apply another interpreter:

In [26]:
fmap view lst_sym

["114","514","(114 + 514)"]

# Extend the languages

## Add a new interpreter

The approaches of adding new interpreters to the languages have be demonstrated above.

## Add  a new syntactic form for expressions

If we want to extend the languages with new syntax, for example, multiplication operation:

### Initial style

As we defined the expression as an algebraic data type before

```hs
data Exp = Lit Int
    | Neg Exp
    | Add Exp Exp
```

To add a new form, we need to append the new variant to the data type:

In [27]:
data Exp = Lit Int
    | Neg Exp
    | Add Exp Exp
    | Mul Exp Exp

We have to re-declare the definition of data type `Exp`, which forces us to adjust or at least re-compile all the codes that directly or indirectly refers to the declaration.

In the basic functional programming languages, it is easy to add new operations on data (i.e. using new-defined functions to interpret expressions in new ways) but hard to add new variants.

That is what frequently discussed in [*the expression problem*](https://en.wikipedia.org/wiki/Expression_problem).

Now, we start to look at how to add multiplication form in the final encoding.

### Final style

Suppose that `ExpSYM` and its two instances are placed inside an independent (already compiled) module called `F` (located in [ExpSYM.hs](./ExpSYM.hs))

In [28]:
:l ExpSYM
import ExpSYM as F

We now want to add multiplication syntax in current module. We only need to define a new type class as:

In [29]:
class MulSYM repr where 
    mul :: repr -> repr -> repr

Extend language terms:

In [30]:
y = mul (lit 364) F.x

Now, we can infer that `y` should be a variable of a type instance that satisfies both 2 type classes.

In [31]:
:type y

y :: forall repr. (MulSYM repr, ExpSYM repr) => repr

To extend the two existing interpreters, we don't need to touch anything in module `F`, including the definitions of `eval` and `view` themselves. 

We are going to define the meaning of `mul` in the semantic domains of type `Int` and `String`:

In [32]:
instance MulSYM Int where
    mul e1 e2 = e1 * e2
    
instance MulSYM String where
    mul e1 e2 = "(" ++ e1 ++ " * " ++ e2 ++ ")"

That is all.

In [33]:
eval y

469924

In [34]:
view y

"(364 * (1919 + (-(114 + 514))))"

# The de-serialization problem

*The de-serialization problem* is spun off the expression problem.

More details can be referenced in [this lecture](https://userpages.uni-koblenz.de/~laemmel/TheEagle/resources/xproblem2.html).

So far, we have created all expressions by directly entering Haskell code, compiled and interpreted on the fly. However, sometimes we need to import/export expressions from/to files.

The two processes are not so symmetric. In general, writing terms into bytes (i.e. serialization) is unproblematic (e.g. pretty-printer), while the converse is much more difficult.

Parsing the sequences which represent an embedded language term should be necessarily *partial*: once upon coming across a parse error in the reading sequence, it returns an error immediately rather than keeping parsing the rest of the input.

Let's begin with a JSON-like format, which represented in Haskell as a `Tree`:

In [35]:
data Tree = Leaf String -- atom element
    | Node String [Tree] -- a collection of elements
    deriving (Eq, Read, Show)

We utilize Haskell built-in `read` and `show` function to read and write `Tree` values from/to files.

The serializer, `toTree` can be simply considered as another interpreter, which evals expressions as `Tree`s and is very similar to `view`.

Therefore, it can be written as:

In [36]:
instance ExpSYM Tree where
    lit n = Node "Lit" [Leaf $ show n]
    neg e = Node "Neg" [e]
    add e1 e2 = Node "Add" [e1,e2]
    
toTree:: Tree -> Tree
toTree = id

Check the converter:

In [41]:
xTree = toTree F.x
xTree

Node "Add" [Node "Lit" [Leaf "1919"],Node "Neg" [Node "Add" [Node "Lit" [Leaf "114"],Node "Lit" [Leaf "514"]]]]

The serialized and nested structure looks like JSON data, or an S-expression.

In de-serialization, our task is to write a function called `fromTree`, which converts a `Tree` to an expression in our final-embedding language.

To start, we have to determine the signature of `fromTree`. For that the type of an expression is typically `ExpSYM repr => repr`, we may consider the function `fromTree` of such a type:

```hs
fromTree :: ExpSYM repr => Tree -> repr
```

Recall that the deserializer should short-circuit and early return an error message when receiving an invalid input. So we turn to the `Error` monad and wrap a `safeRead` function to safely read an `Int` or other readable value, reporting the parser error in time.  

In [38]:
type ErrMsg = String

safeRead :: Read a => String -> Either ErrMsg a
safeRead s = case reads s of
    [(x,"")] -> Right x
    _ -> Left $ "Read error: " ++ s

We can use `safeRead` to de-serialize integer literals (`lit` expression). Then we can de-serialize composite expressions inductively. 

**Note:** `liftM` and `liftM2` are used to lift an operation about`repr` to the one about `Either ErrMsg repr` automatically.

In [39]:
fromTree:: (ExpSYM repr) => Tree -> Either ErrMsg repr
fromTree (Node "Lit" [Leaf n]) = liftM lit $ safeRead n
fromTree (Node "Neg" [e])      = liftM neg $ fromTree e
fromTree (Node "Add" [e1,e2])  = liftM2 add (fromTree e1) (fromTree e2)
fromTree e = Left $ "Invalid tree: " ++ show e

Line 2: Use fmap
Found:
liftM
Why not:
fmapLine 3: Use fmap
Found:
liftM
Why not:
fmap

Now, we try to de-serialize and view the `Tree` we obtained before (as we know that the `Tree` must be valid so we will get a `(Left repr)`):

In [45]:
import Data.Either
putStrLn $ fromRight "parser error" $ fromTree xTree

(1919 + (-(114 + 514)))

If you want to interpret the de-serialized result for many times with corresponding interpreters, for example:

In [50]:
case fromTree xTree of
    Left e -> putStrLn $ "Error: " ++ e
    Right x -> do print $ eval x
                  print $ view x

: 

We catch a type error, reporting that `x` cannot be of both types `Int` and `String`. Here we have lost polymorphism.

The function `fromTree` is indeed polymorphic over `repr` as `(ExpSYM repr) =>` in its signature shows. However, to extract de-serialized `x`, we have to do pattern-matching, in which the variable `x` is bound and hence, like a lambda-pattern-bound variable, gets a monomorphic, non-generalized type. The extensibility is impossible in the solution above.